In [301]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime
from time import mktime
from pprint import pprint

In [302]:
flights_all = pd.read_csv('flights.csv')

In [303]:
flights_all.head()

,source,destination,departure,arrival,flight_number,price,bags_allowed,bag_price
0,USM,HKT,2017-02-11T06:25:00,2017-02-11T07:25:00,PV404,24,1,9
1,USM,HKT,2017-02-12T12:15:00,2017-02-12T13:15:00,PV755,23,2,9
2,USM,HKT,2017-02-12T21:15:00,2017-02-12T22:15:00,PV729,25,1,14
3,USM,HKT,2017-02-11T14:50:00,2017-02-11T15:50:00,PV966,21,1,17
4,USM,HKT,2017-02-12T00:35:00,2017-02-12T01:35:00,PV398,24,1,14


## Utils

In [341]:
# Preprocessing
def date2timestamp(date: str):
    return mktime(datetime.strptime(date, "%Y-%m-%dT%H:%M:%S").timetuple())

def preprocess(flights: DataFrame):
    flights['departure'] = flights['departure'].apply(lambda x: date2timestamp(x))
    flights['arrival'] = flights['arrival'].apply(lambda x: date2timestamp(x))
    return flights

def make_subtree(flight: str, flights: DataFrame):
    sub_tree = {}
    row = flights[flights['flight_number'] == flight]
    time_now, airport_now = row[['arrival', 'destination']].values[0]
    flights_future = flights[flights['departure'] >= time_now + 60*60]              
    flights_now = flights_future[flights_future['departure'] <= time_now + 4*60*60]
    
    if len(flights_now) == 0 or airport_now not in flights_now['source'].values:
        return sub_tree
    else:
        airport_flights_now = flights_now['flight_number'][ flights_now['source'] == airport_now ]
        for flight_now in airport_flights_now:
            sub_tree[flight_now] = make_subtree(flight_now, flights_future)
            
        return sub_tree

# search for possible combination tree
def make_tree(flights: DataFrame, num_bags=0):
    tree = {}
    airports = flights['source'].unique()
    
    for airport in airports:
        tree[airport] = {}
        airport_flights = flights['flight_number'][(flights['source'] == airport) & \
                                                   (flights['bags_allowed'] >= num_bags)]

        for flight in airport_flights:
            sub_tree = make_subtree(flight, flights)
            tree[airport][flight] = sub_tree
            
    return tree

# assuming there are no more than 2 stopovers
def search_combinations(tree: dict):
    combinations_all = []
    for airport in tree:
        for flight in tree[airport]:
            combination = [flight]
            combinations_all += [combination]
            
            for flight_next in tree[airport][flight]:
                combination_next = [flight_next]
                combinations_all += [combination_next]
                combinations_all += [combination + combination_next]
                
                for flight_next_next in tree[airport][flight][flight_next]:
                    combination_next_next = [flight_next_next]
                    combinations_all += [combination_next_next]
                    combinations_all += [combination_next + combination_next_next]
                    combinations_all += [combination + combination_next + combination_next_next]
        
    return combinations_all

def filter_duplicates(combinations: list):
    combinations_unique = set(tuple(x) for x in combinations)
    return [list(x) for x in combinations_unique]

def filter_cycles(combinations: list, flights: DataFrame):
    for combination in combinations:
        if len(combination) > 2: # able to make cycled route
            routes = []
            for flight in combination:
                row = flights[flights['flight_number'] == flight]
                route = row[['source', 'destination']].values[0]
                if list(route) in routes: 
                    combinations.remove(combination)
                    break
                else:
                    routes += [list(route)]
                    
    return combinations

def add_prices(combinations: list, flights: DataFrame, num_bags):
    num_combination = len(combinations)
    for i in range(num_combination):
        combination_price = 0
        for flight in combinations[i]:      
            row = flights[flights['flight_number'] == flight]
            combination_price += row['price'].values[0] + num_bags * row['bag_price'].values[0]
        
        combinations[i] += [combination_price]
        
    return combinations

In [305]:
flights = preprocess(flights_all)
flights.head()

,source,destination,departure,arrival,flight_number,price,bags_allowed,bag_price
0,USM,HKT,1.486791e+09,1.486794e+09,PV404,24,1,9
1,USM,HKT,1.486898e+09,1.486902e+09,PV755,23,2,9
2,USM,HKT,1.486930e+09,1.486934e+09,PV729,25,1,14
3,USM,HKT,1.486821e+09,1.486825e+09,PV966,21,1,17
4,USM,HKT,1.486856e+09,1.486860e+09,PV398,24,1,14


In [335]:
bags = 0
tree = make_tree(flights, num_bags=bags)
print("\n Flights tree with at least {} number of bags allowed:".format(bags))
pprint(tree)


 Flights tree with at least 0 number of bags allowed:
{'BWN': {'PV042': {},
         'PV046': {'PV451': {'PV042': {}}, 'PV974': {'PV672': {}}},
         'PV213': {'PV197': {}},
         'PV278': {},
         'PV378': {'PV414': {'PV243': {}}},
         'PV388': {},
         'PV452': {},
         'PV873': {'PV260': {'PV243': {}}},
         'PV883': {},
         'PV953': {},
         'PV999': {}},
 'DPS': {'PV197': {},
         'PV207': {'PV634': {}},
         'PV260': {'PV243': {}},
         'PV414': {'PV243': {}},
         'PV451': {'PV042': {}},
         'PV478': {},
         'PV519': {'PV442': {}},
         'PV620': {'PV042': {}},
         'PV699': {'PV634': {}},
         'PV974': {'PV672': {}}},
 'HKT': {'PV100': {},
         'PV101': {'PV870': {'PV837': {'PV275': {}, 'PV290': {}, 'PV320': {}}}},
         'PV146': {},
         'PV243': {},
         'PV442': {},
         'PV634': {},
         'PV672': {},
         'PV837': {'PV275': {}, 'PV290': {}, 'PV320': {}},
         'PV961': {}

In [336]:
combinations = search_combinations(tree)
combinations = filter_cycles(combinations, flights)
combinations = filter_duplicates(combinations)
combinations = add_prices(combinations, flights, num_bags=bags)
pprint(combinations)
len(combinations)

[['PV278', 41],
 ['PV414', 67],
 ['PV755', 'PV634', 44],
 ['PV996', 'PV540', 49],
 ['PV378', 59],
 ['PV046', 50],
 ['PV046', 'PV451', 107],
 ['PV260', 'PV243', 111],
 ['PV519', 'PV442', 96],
 ['PV999', 54],
 ['PV876', 'PV442', 42],
 ['PV101', 18],
 ['PV378', 'PV414', 'PV243', 148],
 ['PV996', 23],
 ['PV873', 46],
 ['PV870', 19],
 ['PV953', 48],
 ['PV540', 'PV634', 47],
 ['PV634', 21],
 ['PV388', 49],
 ['PV876', 25],
 ['PV966', 'PV146', 42],
 ['PV442', 17],
 ['PV699', 78],
 ['PV042', 56],
 ['PV378', 'PV414', 126],
 ['PV478', 47],
 ['PV404', 24],
 ['PV046', 'PV974', 'PV672', 159],
 ['PV519', 79],
 ['PV243', 22],
 ['PV837', 18],
 ['PV961', 70],
 ['PV883', 51],
 ['PV966', 21],
 ['PV620', 43],
 ['PV451', 57],
 ['PV620', 'PV042', 99],
 ['PV672', 24],
 ['PV837', 'PV275', 42],
 ['PV275', 24],
 ['PV414', 'PV243', 89],
 ['PV213', 55],
 ['PV213', 'PV197', 105],
 ['PV046', 'PV974', 135],
 ['PV100', 96],
 ['PV101', 'PV870', 37],
 ['PV207', 'PV634', 104],
 ['PV699', 'PV634', 99],
 ['PV320', 22],
 ['

68